In [1]:
from utils.syntax import PREPOSITIONS
import re

In [2]:
# Retain preposition tag
edit_tag = r'(\[-(.+?)-\]|\{\+(.+?)\+\})'
def retain_target(line):
    is_target = False
    matches = re.findall(edit_tag, line)
    for edit, delete, insert in matches:
        if delete in PREPOSITIONS or insert in PREPOSITIONS: is_target = True
        elif delete: line = line.replace(edit, '')
        elif insert: line = line.replace(edit, insert)
    return line if is_target else None


# Transform format [-for-] {+in+} >> [-for>>in+}
replace_format = r'(\[-([^-]+?)-\] \{\+([^+]+?)\+\})'
def transform_replace(line):
    matches = re.findall(replace_format, line)
    for edit, delete, insert in matches:
        line = line.replace(edit, "[-{}>>{}+}}".format(delete, insert))
    return line


# One line one error
new_edit_tag = r'(\[-(.+?)-\]|{\+(.+?)\+}|\[-(.+?)>>(.+?)\+})'
def duplicate(line):
    dups = []
    matches = re.findall(new_edit_tag, line)
    for i in range(len(matches)):
        dup = line
        for j, (edit, delete, insert, rd, ri) in enumerate(matches):
            if   i == j: continue
            elif delete: dup = dup.replace(edit, '')
            elif insert: dup = dup.replace(edit, insert)
            else:        dup = dup.replace(edit, ri)
        dups.append(dup)
    return dups

In [13]:
lines = open('/atom/corpus/written/EFCAM/EFCAM.error.txt', 'r', encoding='utf8', errors='ignore').readlines()

remain_lines = []
for line in lines[100:110]:
    line = retain_target(line.strip())
    if line:
        line = transform_replace(line.replace('  ', ' '))
        dups = duplicate(line)
        print(dups)
        remain_lines.extend(dups)

An area of about 8 meters [-x-] by 3 meters is marked as the ' bowling alley '.

["An area of about 8 meters [-x-] by 3 meters is marked as the ' bowling alley '.\n"]
Fill {+them+} with a little water to make them heavier .

['Fill {+them+} with a little water to make them heavier .\n']
Line {+them+} up in rows of 4 , then , 3 , then 2 [-,-] {+and+} then 1 - just like {+in+} [-ten-] {+Ten+} - pin [-bowling-] {+Bowling+} .

['Line {+them+} up in rows of 4 , then , 3 , then 2 and then 1 - just like in Ten - pin Bowling .\n', 'Line them up in rows of 4 , then , 3 , then 2 [-,>>and+} then 1 - just like in Ten - pin Bowling .\n', 'Line them up in rows of 4 , then , 3 , then 2 and then 1 - just like {+in+} Ten - pin Bowling .\n', 'Line them up in rows of 4 , then , 3 , then 2 and then 1 - just like in [-ten>>Ten+} - pin Bowling .\n', 'Line them up in rows of 4 , then , 3 , then 2 and then 1 - just like in Ten - pin [-bowling>>Bowling+} .\n']
Give a [-frisbee-] {+Frisbee+} to each player and 

In [10]:
# ws = open('wikedit_prep', 'w', encoding='utf8')
# for line in remain_lines:
#     print(line, file=ws)
# ws.close()

In [6]:
remain_lines

['15 sq with an extension of up to 150 sqm permissible with a price of $200,000.',
 '15 sq with an extension of up to 150 sqm permissible with a price of $200,000.',
 'It is much larger than the first one , the land  is 1200 sqm approx and the house  is 224 .',
 'It is much larger than the first one , the land  is 1200 sqm approx and the house  is 224 .',
 'As he was approaching the corner at Elm and Fourth , he saw a young couple get out of a car and talk about noise and air pollution [-with>>by+} leaving the engine running and the stereo playing .']

In [4]:
short_lines = list(filter(lambda line: len(line.split(' ')) <= 25, remain_lines))

In [5]:
short_lines[:10]

['15 sq with an extension of up to 150 sqm permissible with a price of $200,000.',
 '15 sq with an extension of up to 150 sqm permissible with a price of $200,000.',
 'It is much larger than the first one , the land  is 1200 sqm approx and the house  is 224 .',
 'It is much larger than the first one , the land  is 1200 sqm approx and the house  is 224 .']